In [3]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

In [659]:
cluster = Cluster(ClusterConfiguration(
    name='gptfttest',
    namespace='default',
    min_worker=2,
    max_worker=2,
    min_cpus=2,
    max_cpus=2,
    min_memory=8,
    max_memory=8,
    gpu=1,
    instascale=True,
    machine_types=["m5.xlarge", "g4dn.xlarge"],
))

Written to: gptfttest.yaml


In [660]:
cluster.up()

In [661]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


In [662]:
cluster.details()

                   🚀 CodeFlare Cluster Details 🚀                  
                                                                    
 ╭────────────────────────────────────────────────────────────────╮ 
 │   Name                                                         │ 
 │   gptfttest                                        Active ✅   │ 
 │                                                                │ 
 │   URI: ray://gptfttest-head-svc.default.svc:10001              │ 
 │                                                                │ 
 │   ]8;id=618416;http://ray-dashboard-gptfttest-default.apps.meyceoz-032023.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                  │ 
 │                                                                │ 
 │                      Cluster Resources                         │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮     │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │     │ 
 │   │            │  │                                      │     │ 
 │   │  2    2    │  │  8~8         2           1           │     │ 
 │   │            │  │                                      │     │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯     │ 
 ╰────────────────────────────────────────────────────────────────╯

RayCluster(name='gptfttest', status=<RayClusterStatus.READY: 'ready'>, min_workers=2, max_workers=2, worker_mem_min=8, worker_mem_max=8, worker_cpu=2, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-gptfttest-default.apps.meyceoz-032023.psap.aws.rhperfscale.org')

In [663]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [664]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--do_train",
    "--do_eval",
    "--output_dir", "/tmp/test-clm",
    "--overwrite_output_dir"
]

In [665]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="gpt_og.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements_gpt.txt"}
)
job = jobdef.submit(cluster)

The Ray scheduler does not support port mapping.


In [675]:
job.status()

AppStatus:
  msg: !!python/object/apply:ray.dashboard.modules.job.common.JobStatus
  - SUCCEEDED
  num_restarts: -1
  roles:
  - replicas:
    - hostname: <NONE>
      id: 0
      role: ray
      state: !!python/object/apply:torchx.specs.api.AppState
      - 4
      structured_error_msg: <NONE>
    role: ray
  state: SUCCEEDED (4)
  structured_error_msg: <NONE>
  ui_url: null

In [674]:
job.logs()

'[RayActor(name=\'gpt_og\', command=[\'bash\', \'-c\', "torchrun --rdzv_backend static --rdzv_endpoint $TORCHX_RANK0_HOST:49782 --rdzv_id \'gpttest-czbl5zhq5hnl7c\' --nnodes 2 --nproc_per_node 1 --node_rank \'0\' --tee 3 --role \'\' gpt_og.py --model_name_or_path gpt2 --dataset_name wikitext --dataset_config_name wikitext-2-raw-v1 --per_device_train_batch_size 2 --per_device_eval_batch_size 2 --do_train --do_eval --output_dir /tmp/test-clm --overwrite_output_dir"], env={\'TORCHX_TRACKING_EXPERIMENT_NAME\': \'default-experiment\', \'LOGLEVEL\': \'WARNING\', \'TORCHX_JOB_ID\': \'ray://torchx/gpttest-czbl5zhq5hnl7c\'}, num_cpus=2, num_gpus=1, min_replicas=2), RayActor(name=\'gpt_og\', command=[\'bash\', \'-c\', "torchrun --rdzv_backend static --rdzv_endpoint $TORCHX_RANK0_HOST:49782 --rdzv_id \'gpttest-czbl5zhq5hnl7c\' --nnodes 2 --nproc_per_node 1 --node_rank \'1\' --tee 3 --role \'\' gpt_og.py --model_name_or_path gpt2 --dataset_name wikitext --dataset_config_name wikitext-2-raw-v1 --pe

In [658]:
cluster.down()

In [ ]:
auth.logout()